# Emotion Chat Bot

In [1]:
from pprint import pprint

import torch
from libs import (
    EmotionModel,
    ResponseGeneratorPipeline,
    SimilarityAnalyser,
    generate_dummy_representation,
    get_sentiment_composition,
)
from sympy.core.random import randint
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)
from unsloth import FastLanguageModel

from src.models.rg_evaluate_sft import result

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load Each Module

### Response Generator

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


In [3]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [4]:
# response_generator.model = torch.compile(response_generator.model, mode="max-autotune")
FastLanguageModel.for_inference(response_generator.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32005, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-0

In [5]:
print(response_generator.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32005, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-0

### Sentiment Analyzer

In [6]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [7]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [8]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [9]:
sentiment_analyzer.model = torch.compile(sentiment_analyzer.model, mode="max-autotune")

In [10]:
print(sentiment_analyzer.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Predictor

In [11]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [12]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [13]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [14]:
emotion_predictor.model = torch.compile(emotion_predictor.model, mode="max-autotune")

In [15]:
print(emotion_predictor.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Model

In [16]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [17]:
emotion_model = torch.compile(emotion_model, mode="max-autotune")

In [18]:
print(emotion_model)

OptimizedModule(
  (_orig_mod): EmotionModel(
    (_EmotionModel__weight_Q): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__weight_K): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__dropout): Dropout(p=0.5, inplace=False)
    (_EmotionModel__weight_D): Linear(in_features=7, out_features=7, bias=True)
  )
)


### Similarity Analyzer

In [19]:
threshold: float = 0.5

In [20]:
similarity_analyzer = SimilarityAnalyser(threshold)

## Combine All Modules

### Initialize Text Streamer

In [34]:
streamer = TextStreamer(
    tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [71]:
system_prompt: str = input("Enter your system prompt: ").strip()

In [82]:
chat_buffer: list = [
    {
        "role": "system",
        "content": {"emotion": "", "dialog": system_prompt},
    }
]

### Initialize Bot's Emotion Representation

In [83]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

tensor([0.6998, 0.3871, 0.0518, 0.8533, 0.6769, 0.6137, 0.5486])

In [84]:
def get_top_emotion(input_text_emotion: list) -> str:
    label2score: dict = {
        emotion["label"]: emotion["score"] for emotion in input_text_emotion[0]
    }
    return max(label2score, key=label2score.get)

In [85]:
input_text: str = input("Talk to bot...").strip()
input_text_emotion: list = sentiment_analyzer(chat_buffer[-1]["content"]["dialog"])
chat_buffer.append(
    {
        "role": "user",
        "content": {
            "emotion": get_top_emotion(input_text_emotion),
            "dialog": input_text,
        },
    }
)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}}]

In [86]:
input_text_emotion_composition: Tensor = get_sentiment_composition(input_text_emotion)
input_text_emotion_composition

tensor([0.3073, 0.1151, 0.1151, 0.1151, 0.1168, 0.1153, 0.1152])

### Update Current Bot's Emotion Representation

In [87]:
bot_emotion_representation = emotion_model.forward(
    input_text_emotion_composition, bot_emotion_representation
)
bot_emotion_representation

tensor([0.4908, 0.2974, 0.2981, 0.2969, 0.2986, 0.2971, 0.2979],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)

### Generate All Possible Responses In 7 Different Emotions

In [90]:
def create_candidates_buffer(chat_buffer: list) -> list:
    emotions: list = [
        "neutral",
        "anger",
        "disgust",
        "fear",
        "happiness",
        "sadness",
        "surprise",
    ]

    candidates_buffer: list = [
        chat_buffer
        + [
            {
                "role": "bot",
                "content": {"emotion": emotion, "dialog": ""},
            }
        ]
        for emotion in emotions
    ]

    return candidates_buffer

In [91]:
candidates_buffer: list = create_candidates_buffer(chat_buffer)
candidates_buffer

[[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'neutral', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'anger', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'disgust', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'fear', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 

In [106]:
generated_candidates: list = [
    result[0]["generated_text"]
    for result in response_generator(candidates_buffer, streamer=streamer)
    if result[0]["generated_text"][-1]["content"]["dialog"] != ""
]
print()
pprint(generated_candidates)

   neutral   Hello, how are you?     neutral    Fine, thank you. 
   neutral   Hello, how are you?     anger   
   neutral   Hello, how are you?     disgust   
   neutral   Hello, how are you?     fear    Hi! I'm terribly nervous. 
   neutral   Hello, how are you?     happiness     Fine, thank you. 
   neutral   Hello, how are you?     sadness   
   neutral   Hello, how are you?     surprise    hello! I haven't seen you for ages! 

[[{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': ' Fine, thank you. ', 'emotion': 'neutral'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': " Hi! I'm terribly nervous. ", 'emotion': 'fear'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'

### Simulate Bot's Possible Emotion Representation

In [ ]:
# def get_possible_response_emotion_representation(candidates_buffer: list) -> list:
# 	for

In [ ]:
# possible_response_emotion_representation: dict =